## Oxylabs Search Run

This notebook goes over how to use the Oxylabs search run component.


### Integration details


| Package                                                          |
|------------------------------------------------------------------|
| [langchain-oxylabs](https://pypi.org/project/langchain-oxylabs/) |

### Tool features

| [Returns artifact](/docs/how_to/tool_artifacts/) | Pricing         |
| :---: | :-----------------------:|
| ✅ | Free 5,000 results for 1 week |

## Setup

Install the required dependencies.

In [ ]:
%pip install -U langchain-oxylabs

### Credentials

Set up the proper API keys and environment variables.  
Create your API user credentials: Sign up for a free trial or purchase the product
in the [Oxylabs dashboard](https://dashboard.oxylabs.io/en/registration)
to create your API user credentials (OXYLABS_USERNAME and OXYLABS_PASSWORD).

In [ ]:
import getpass
import os

os.environ["OXYLABS_USERNAME"] = getpass.getpass()
os.environ["OXYLABS_PASSWORD"] = getpass.getpass()

## Instantiation

In [ ]:
from langchain_oxylabs import OxylabsSearchRun, OxylabsSearchAPIWrapper

oxylabs_wrapper = OxylabsSearchAPIWrapper()

In [ ]:
tool_ = OxylabsSearchRun(wrapper=oxylabs_wrapper)

## Invocation

### [Invoke directly with args](/docs/concepts/#invoke-with-just-the-arguments)

The `OxylabsSearchRun` tool takes a single "query" argument,  
which should be a natural language query and returns combined string format result:

In [ ]:
tool_.invoke({"query": "Visit restaurants in Vilnius."})

### [Invoke with ToolCall](/docs/concepts/#invoke-with-toolcall)

In [ ]:
tool_ = OxylabsSearchRun(
    wrapper=oxylabs_wrapper,
    kwargs={
        "result_categories": [
            "local_information",
            "combined_search_result",
        ]
    },
)

In [ ]:
from pprint import pprint

model_generated_tool_call = {
    "args": {
        "query": "Visit restaurants in Vilnius.",
        "geo_location": "Vilnius,Lithuania",
    },
    "id": "1",
    "name": "oxylabs_search",
    "type": "tool_call",
}
tool_call_result = tool_.invoke(model_generated_tool_call)

# The content is a JSON string of results
pprint(tool_call_result.content)

## Chaining

In [ ]:
# Install the required libraries
%pip install --upgrade --quiet langchain langchain-openai
%pip install langchainhub langchain-community

In [ ]:
import getpass
import os

from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_openai import ChatOpenAI

# Set up OpenAI API credentials
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

# Define assistant instructions and pull a base prompt template
instructions = """You are an assistant."""
base_prompt = hub.pull("langchain-ai/openai-functions-template")
prompt = base_prompt.partial(instructions=instructions)

# Initialize OpenAI chat model
llm = ChatOpenAI(openai_api_key=os.environ["OPENAI_API_KEY"])

# Define tools and agent setup
tool_ = OxylabsSearchRun(wrapper=oxylabs_wrapper)
tools_ = [tool_]
agent = create_tool_calling_agent(llm, tools_, prompt)

# Set up and invoke the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools_,
    verbose=True,
)
agent_executor.invoke({"input": "What happened in the latest Burning Man floods?"})

In [ ]:
agent_executor.invoke(
    {"input": "What is the most profitable company in Lithuania in 2024?"}
)